## Working with data

In [14]:
import torch
from torch import nn
from torch.utils.data import DataLoader # wraps iterable around dataset
from torchvision import datasets # stores samples and labels
from torchvision.transforms import ToTensor

In [15]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Pass `Dataset` to dataloader, which wraps it in an iterable. This supports batching, shuffling, and sampling

In [16]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X,y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models

Define models by creating classes that inherit from `nn.Module`. Define the layers of the network in `__init__` and forward pass in `forward`. Use CUDA, MPS, MTIA if available, otherwise use CPU.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [23]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [25]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters

To train a model, need loss function and optimizer

In [26]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

During training, the model makes predictions on the training dataset and backpropogates error to adjust model parameters

In [27]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropogation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [28]:
# Check the model's performance against the test dataset to ensure it is learning
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Training process is carried out in iteration "epochs". Loss should decrease and accuracy should increase per epoch

In [29]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.296513 [   64/60000]
loss: 2.283098 [ 6464/60000]
loss: 2.257901 [12864/60000]
loss: 2.254544 [19264/60000]
loss: 2.236394 [25664/60000]
loss: 2.202948 [32064/60000]
loss: 2.213705 [38464/60000]
loss: 2.164130 [44864/60000]
loss: 2.172515 [51264/60000]
loss: 2.141840 [57664/60000]
Test Error: 
 Accuracy: 46.3%, Avg loss: 2.131214 

Epoch 2
-------------------------------
loss: 2.143758 [   64/60000]
loss: 2.129476 [ 6464/60000]
loss: 2.062798 [12864/60000]
loss: 2.083358 [19264/60000]
loss: 2.021390 [25664/60000]
loss: 1.963023 [32064/60000]
loss: 1.989786 [38464/60000]
loss: 1.891632 [44864/60000]
loss: 1.910652 [51264/60000]
loss: 1.841244 [57664/60000]
Test Error: 
 Accuracy: 54.0%, Avg loss: 1.833799 

Epoch 3
-------------------------------
loss: 1.865423 [   64/60000]
loss: 1.834479 [ 6464/60000]
loss: 1.711535 [12864/60000]
loss: 1.761436 [19264/60000]
loss: 1.646076 [25664/60000]
loss: 1.606358 [32064/60000]
loss: 1.627074 [38464/

## Saving Models

Common way to save model is to save state containing model parameters

In [30]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Loading Models

Recreate models by loading structure and state dictionary

In [31]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [33]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle Boot"
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle Boot", Actual: "Ankle Boot"
